<h1>Trabalho final de Inteligência Artificial</h1>

<h5>Iarah Gonçalves de Almeida</h5>
<h5>Paulo Augusto Borges de Matos</h5>
<h5>David Walter Jansen</h5>

<h3>Bibliotecas necessárias</h3>

In [403]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

<h3>Carregando a base de dados</h3>

As 17 características extraídas dos experimentos <b>c1</b>, <b>c4</b> e <b>c6</b> foram armazenadas em arquivos no formato .mat do Matlab (<i>x1_17.mat</i>, <i>x4_17.mat</i> e <i>x6_17.mat</i>), bem como as medições do desgaste máximo dos experimentos (<i>yt1.mat</i>, <i>yt4.mat</i> e <i>yt6.mat</i>). Em seguida, os arquivos .mat foram convertidos para o formato .csv através de um script do Matlab e, então, carregados através do Pandas. O nome das colunas da base de dados também é definida.

In [404]:
names = ['rmsFX', 'maxFX', 'stdFX', 'rmsFY', 'maxFY', 'stdFY', 'rmsFZ', 'maxFZ', 'stdFZ', 'rmsVX', 'kVX', 'rmsVY', 'kVY', 'rmsVZ', 'kVZ', 'meanEA', 'stdEA']
target = ['maxDesg']
df1 = pd.read_csv('x1.csv', names = names)
df4 = pd.read_csv('x4.csv', names = names)
df6 = pd.read_csv('x6.csv', names = names)
df_yt1 = pd.read_csv('yt1.csv', names = target)
df_yt4 = pd.read_csv('yt4.csv', names = target)
df_yt6 = pd.read_csv('yt6.csv', names = target)
print(df1.shape);
print(df4.shape);
print(df6.shape);
print(df_yt1.shape);
print(df_yt4.shape);
print(df_yt6.shape);

(315, 17)
(315, 17)
(315, 17)
(315, 1)
(315, 1)
(315, 1)


Inclusão da coluna de desgaste máximo nos DataFrames dos experimentos.

In [436]:
df1['maxDesg'] = df_yt1['maxDesg']
df4['maxDesg'] = df_yt4['maxDesg']
df6['maxDesg'] = df_yt6['maxDesg']

In [437]:
df1.apply(pd.to_numeric).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 18 columns):
rmsFX      315 non-null float64
maxFX      315 non-null float64
stdFX      315 non-null float64
rmsFY      315 non-null float64
maxFY      315 non-null float64
stdFY      315 non-null float64
rmsFZ      315 non-null float64
maxFZ      315 non-null float64
stdFZ      315 non-null float64
rmsVX      315 non-null float64
kVX        315 non-null float64
rmsVY      315 non-null float64
kVY        315 non-null float64
rmsVZ      315 non-null float64
kVZ        315 non-null float64
meanEA     315 non-null float64
stdEA      315 non-null float64
maxDesg    315 non-null float64
dtypes: float64(18)
memory usage: 44.4 KB


In [438]:
df4.apply(pd.to_numeric).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 18 columns):
rmsFX      315 non-null float64
maxFX      315 non-null float64
stdFX      315 non-null float64
rmsFY      315 non-null float64
maxFY      315 non-null float64
stdFY      315 non-null float64
rmsFZ      315 non-null float64
maxFZ      315 non-null float64
stdFZ      315 non-null float64
rmsVX      315 non-null float64
kVX        315 non-null float64
rmsVY      315 non-null float64
kVY        315 non-null float64
rmsVZ      315 non-null float64
kVZ        315 non-null float64
meanEA     315 non-null float64
stdEA      315 non-null float64
maxDesg    315 non-null float64
dtypes: float64(18)
memory usage: 44.4 KB


In [439]:
df6.apply(pd.to_numeric).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 18 columns):
rmsFX      315 non-null float64
maxFX      315 non-null float64
stdFX      315 non-null float64
rmsFY      315 non-null float64
maxFY      315 non-null float64
stdFY      315 non-null float64
rmsFZ      315 non-null float64
maxFZ      315 non-null float64
stdFZ      315 non-null float64
rmsVX      315 non-null float64
kVX        315 non-null float64
rmsVY      315 non-null float64
kVY        315 non-null float64
rmsVZ      315 non-null float64
kVZ        315 non-null float64
meanEA     315 non-null float64
stdEA      315 non-null float64
maxDesg    315 non-null float64
dtypes: float64(18)
memory usage: 44.4 KB


Trabalhando as bases de dados dos 3 experimentos.

### Seleção de características com árvore de decisão
### Passos
1. Separação entre conjuntos de treinamento e teste. (i, j, z representam permutações na escolhas para as bases 1, 4 e 6)  
    1.1 Treinamento: Ci + Cj  
    1.2 Teste: Cz  
2. Aplicação da Random Forest k-vezes.  
    2.1 Para cada k, utilizar um sistema de pontuação para as características mais importantes.  
    2.2 Criar ordenação geral das características, quanto a sua importânca.  
3. Utilização mapa de calor para exibir as coorelações entre as características.  
    3.1 Criar uma lista de pares correlacionados para reduzir características usadas. Utilizar a ordenação das características como critério para remoção.
4. Avaliação do resultado.  
    4.1 Gerar nova regressão utilizando conjunto reduzido de características.  
    4.2 Comparar resultados.
    
**Obs:** O passo 3 pode ser expandido usando combinações de características para gerar novas

<h4>Random Forest Regressor</h4>

<p><b>OBS.:</b> Nessa etapa, usaremos o seguinte alinhamento: O i-ésimo conjunto de características X, terá o target como o (i+1)-ésimo Y, indicando que a regressão será dada para um próximo passo.</p>

In [441]:
# Isola as 17 características
X1 = df1.iloc[:-1,0:17]
X4 = df4.iloc[:-1,0:17]
X6 = df6.iloc[:-1,0:17]

# Escolhe o desgaste máximo
Y1 = df1.iloc[1:,-1]
Y4 = df4.iloc[1:,-1]
Y6 = df6.iloc[1:,-1]

# Seleciona o nome das 17 características
names = df1.columns.values
names = names[:17]

In [442]:
# Usaremoas df1 e df4 para treino e df6 para validar esse treinamento
X_train = X4.append(X6, ignore_index = True, sort = False)
Y_train = Y4.append(Y6, ignore_index = True)

X_test = X1
Y_test = Y1
# Constrói o modelo de regressão com 100 árvores na floresta aleatória
regressor = RandomForestRegressor(n_estimators = 100)

# Importância das características
importance = 0
Score = 0
for i in range(10):
    # Ajusta o modelo de regressão
    regressor.fit(X_train, Y_train)
    Y_pred = regressor.predict(X_test)
    # Soma as importâncias das características (quanto maior, mais importante)
    importance = importance + regressor.feature_importances_
    Score =  Score + regressor.score(X_test, Y_test)

Score = Score/(i+1) # médica
print("Precisão: ", end=" ")
print(Score)

Precisão:  0.676454139938008


In [443]:
# Usaremoas df1 e df4 para treino e df6 para validar esse treinamento
X_train = X1.append(X4, ignore_index = True, sort = False)
Y_train = Y1.append(Y4, ignore_index = True)

X_test = X6
Y_test = Y6

# Constrói o modelo de regressão com 100 árvores na floresta aleatória
regressor = RandomForestRegressor(n_estimators = 100)

# Importância das características
importance = 0
Score = 0
for i in range(10):
    # Ajusta o modelo de regressão
    regressor.fit(X_train, Y_train)
    Y_pred = regressor.predict(X_test)
    # Soma as importâncias das características (quanto maior, mais importante)
    importance = importance + regressor.feature_importances_
    Score =  Score + regressor.score(X_test, Y_test)

Score = Score/(i+1) # médica
print("Precisão: ", end=" ")
print(Score)

Precisão:  0.7113867269722169


In [444]:
# Usaremoas df1 e df4 para treino e df6 para validar esse treinamento
X_train = X1.append(X6, ignore_index = True, sort = False)
Y_train = Y1.append(Y6, ignore_index = True)

X_test = X4
Y_test = Y4
# Constrói o modelo de regressão com 100 árvores na floresta aleatória
regressor = RandomForestRegressor(n_estimators = 100)

# Importância das características
importance = 0
Score = 0
for i in range(10):
    # Ajusta o modelo de regressão
    regressor.fit(X_train, Y_train)
    Y_pred = regressor.predict(X_test)
    # Soma as importâncias das características (quanto maior, mais importante)
    importance = importance + regressor.feature_importances_
    Score =  Score + regressor.score(X_test, Y_test)

Score = Score/(i+1) # médica
print("Precisão: ", end=" ")
print(Score)

Precisão:  0.7726756932233568
